# Introduction to Time Series

**Time series** data is a distinctive form of statistical information, representing a sequence of measurements ${x_1}$, ${x_2}$, ... ${x_n}$ indexed by timestamps t = 1, 2, ... n. These timestamps constitute a deterministic sequence with equal intervals between adjacent points. In statistical terms, we model the observations ${x_1}$, ${x_2}$, ... ${x_n}$ as realizations of a series of **random variables** ${X_1}$, ${X_2}$, ... ${X_n}$.

A defining feature of time series is the absence of assumptions regarding the independence of these random variables. More often than not, they exhibit dependence due to the influence of past observations on future ones, reflecting real-world phenomena. The probabilistic model encapsulating such a phenomenon is termed a **stochastic process**. Hence, time series data can be viewed as a single realization of a stochastic process.

Central to time series analysis are dependencies such as **trend** and **seasonal variation**. These represent deterministic relationships of the random variable ${X_t}$ with the timestamp t. Considering the regression of the time series on the time index, we denote the mean function of the marginal distribution as:
$$
\begin{equation}
 {μ_X}(t) = E[{X_t}]
\end{equation}
$$
Here, ${μ_X}(t)$ encapsulates both the trend and seasonal variation. The trend (${m_X(t)}$ is the non-constant, non-cyclical component—monotonically evolving with time. Simultaneously, the seasonal variation ${s_X(t)}$ is the cyclical component, a periodic function with values repeating at fixed intervals. Thus, we have the deterministic dependence structure:
$$
\begin{equation}
 {μ_X}(t) = {m_X(t)} + {s_X(t)}
\end{equation}
$$
The primary stochastic feature of a random variable lies in the statistical dependence of variables at different time points, resulting in a joint distribution distinct from the product of marginal distributions. **Autocovariance** and **marginal variance** functions capture these dependencies, quantifying linear associations and the magnitude of random fluctuations, respectively.

The objective of time series analysis is to comprehend and leverage the **deterministic** and **stochastic dependencies** within the underlying stochastic process. This involves detecting trends, identifying seasonal variations, understanding correlation structures within the time series, and exploring correlations between different time series.

In many applications, the ultimate goal is forecasting future observations. Achieving this requires estimating trends and seasonality and fitting a statistical model that captures dependencies. Prediction is then facilitated by extrapolating these learned dependencies.

To draw inferences from time series data, certain technical conditions about the underlying data-generating process must be satisfied. Specifically, we seek conditions ensuring that observations along a single realization are representative of all possible realizations. Introducing the concept of **stationarity** — which means the joint distribution remains consistent across time — becomes crucial. A time series is strongly stationary if the joint distribution of ${X_t}$, ...${X_{t+n}}$ is the same as that of ${X_{t+h}}$, ...${X_{t+n+h}}$ for all n, t, h. Stationarity rules out dependencies on the time index, including trends and seasonality. Consequently, the initial step in the analysis often involves transforming the data into a stationary series, typically by removing trend and seasonal components through regression methods or other techniques.

#Time series models
Let's explore a concise overview of some of the most valuable time series models.


**White Noise:**

*Characteristics:*
Consists of independent and identically distributed (i.i.d.) random variables.
No discernible pattern or correlation between observations.
Constant mean and variance.

**Autoregressive (AR):**

*Characteristics:*
Future values are a linear combination of past observations.
Often denoted as AR(p) where 'p' is the order of the autoregressive process.
Exhibits a gradual decrease in correlation as the lag between observations increases.

**Random Walk:**

*Characteristics:*
Successive observations are dependent, each influenced by the previous one.
No inherent periodicity or seasonality.
Commonly used to model stock prices.

**Moving Average (MA):**

*Characteristics:*
Future values are a linear combination of past white noise error terms.
Often denoted as MA(q) where 'q' is the order of the moving average process.
Exhibits a sudden drop in correlation after a certain lag.

**Autoregressive Integrated Moving Average (ARMA):**

*Characteristics:*
Combines autoregressive (AR) and moving average (MA) components.
Denoted as ARMA(p, q) where 'p' is the AR order and 'q' is the MA order.
Useful for capturing both short-term and long-term dependencies in time series data.

Each of these models serves a different purpose based on the underlying patterns and dependencies present in the data.
While these models represent foundational concepts in time series analysis, it's important to note that they are not state-of-the-art. However, they serve as crucial building blocks, providing a solid understanding that forms the basis for more advanced models often built on top of them.

Now, having provided a brief introduction to time series and highlighted some key models, let's delve into the practical aspects of analysis and forecasting. To illustrate these concepts, we will utilize the 'Electricity Consumption UK 2009-2023' dataset from [Kaggle](https://www.kaggle.com/datasets/albertovidalrod/electricity-consumption-uk-20092022?select=historic_demand_year_2023.csv).

Our primary objective is to thoroughly analyze the time series data, identifying its primary components and fitting a suitable statistical model. To achieve this, we will utilize the dataset inclusive of the year 2022. Subsequently, we will employ this model to predict the consumption for the year 2023, enabling us to compare the forecasted results with the actual data.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from scipy.interpolate import interp1d
from sklearn.metrics import mean_squared_error
%matplotlib inline
sns.set()
pd.set_option('mode.chained_assignment', 'warn')
def mean_absolute_percentage_error(y_true, y_pred):
     return np.mean(np.abs((y_true - y_pred) / y_true)) *100

In [11]:
raw_data = pd.read_csv('historic_demand_2009_2023.csv', index_col = 0)
raw_data.head()

,settlement_date,settlement_period,nd,tsd,england_wales_demand,embedded_wind_generation,embedded_wind_capacity,embedded_solar_generation,embedded_solar_capacity,non_bm_stor,pump_storage_pumping,ifa_flow,ifa2_flow,britned_flow,moyle_flow,east_west_flow,nemo_flow,nsl_flow,eleclink_flow,is_holiday
0,2009-01-01,1,37910,38704,33939,54,1403,0,0,0,33,2002,0,0,-161,0,0,NaN,NaN,1
1,2009-01-01,2,38047,38964,34072,53,1403,0,0,0,157,2002,0,0,-160,0,0,NaN,NaN,1
2,2009-01-01,3,37380,38651,33615,53,1403,0,0,0,511,2002,0,0,-160,0,0,NaN,NaN,1
3,2009-01-01,4,36426,37775,32526,50,1403,0,0,0,589,1772,0,0,-160,0,0,NaN,NaN,1
4,2009-01-01,5,35687,37298,31877,50,1403,0,0,0,851,1753,0,0,-160,0,0,NaN,NaN,1


In [12]:
# Get the last month
raw_data.settlement_date.max()

# Drop November data (at the moment of this analysis November is not over yet)
raw_data.drop(raw_data[raw_data['settlement_date'] >= '2023-11-01'].index, inplace = True)

Check the full column descriptions in the kaggle link above. Here we're only interested in the settlement_date, settlement_period and nd collumns.
**settlement_date**: : date in format dd/mm/yyyy

**settlement_period**: half hourly period for the historic outturn occurred

**nd**: National Demand is the sum of metered generation, but excludes generation required to meet station load, pump storage pumping and interconnector exports. National Demand is calculated as a sum of generation based on National Grid ESO operational generation metering. Measured in MW

# Time series analysis

Before starting the analysis, our first task involves transforming the half-hourly consumption data into a monthly format, laying the groundwork for the subsequent analysis and forecasting.

In [13]:
# Convert the 'settlement_date' to a datetime object, extracting the month's start date
raw_data['month_start'] = pd.to_datetime(raw_data['settlement_date'].str[:7] + '-01', format='%Y-%m-%d')

# Group the data by the start date of each month and sum the 'nd' column to get total monthly consumption
data = (
    raw_data.groupby(['month_start'])['nd']
    .sum()
    .reset_index()
    .rename(columns={'nd': 'consumption'})
)

# Create a new column 'month' that stores the numeric month value
data['month'] = data['month_start'].dt.month

data.head()

,month_start,consumption,month
0,2009-01-01,63238939,1
1,2009-02-01,56073378,2
2,2009-03-01,56519297,3
3,2009-04-01,49311414,4
4,2009-05-01,48437067,5


In [14]:
# Generate summary statistics for the 'consumption' column
data.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,consumption,month
count,178.00000,178.00000
mean,46053830.25843,6.44382
std,7215049.83216,3.43950
min,32156997.00000,1.00000
25%,40618319.00000,3.25000
50%,46267105.00000,6.00000
75%,50773891.50000,9.00000
max,65015253.00000,12.00000


In [15]:
# Create a scatter plot to visualize electricity consumption over the entire time period
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['month_start'],
                         y=data['consumption'],
                         mode='markers',
                         line=dict(color='DarkBlue', width=3)))
fig.update_layout(title='Electricity consumption in UK',
                  xaxis_title='Month',
                  yaxis_title='Consumption')
fig.show()

Two things that we can notice here is that the consumption has decreased over the years (roughly 27% between Jan 2009 and Jan 2023) and there is a seasonal variation in the data: the electricity consumption peaks during the winter months and lowers in the summer months.
Let's assume ${C_i}$ is the electricity consumption in month i (i = 1, 2, ....), we'll try to represent ${C_i}$ in the following form:
$$
\begin{equation}
 {C_i} = F(t_i) + P_i + R_i
\end{equation}
$$
where:

${F(t_i)}$ accounts for the long-term trend

${t_i}$ is the time at the middle of the month i measured in the fractions of years starting from Jan 2009

${P_i}$ is periodic is i with a fixed period, accounting for the seasonal pattern

${R_i}$ is the remaining residual that accounts for all other influences.

The first step is to calculate ${t_i}

 > We convert from $i$ (the data index, starting at 0) to $t_i$ as $t_i = \frac{i + 0.5}{12}$
* $+ 0.5$ because $i$ starts at 0 but the first measurement should be 0.5 (halfway through the first month)
* $/ 12$ to get years from months

In [16]:
# Calculate ti as the time at the middle of each month in fractions of years
data['time'] = (data.index.values + 0.5) / 12

# Assign X as the calculated time and y as the consumption
X = data['time']
y = data['consumption']

print(X[:5])
print(y[:5])

0    0.041667
1    0.125000
2    0.208333
3    0.291667
4    0.375000
Name: time, dtype: float64
0    63238939
1    56073378
2    56519297
3    49311414
4    48437067
Name: consumption, dtype: int64


We want to decompose the time series into its constituent components: trend, seasonality, and residuals. But before diving into the trend decomposition process, let's first partition the dataset into two sets — the training set covering the period from 2009 to 2022, and the test set for the year 2023.

In [17]:
# Find the index corresponding to the start of the year 2023
train_size = len(data[data['month_start'] < '2023-01-01'])

# Display the size of the training set
print('train_size:', train_size)

# Split the time variable (X) and consumption (y) into training and testing sets
X_train = X[:train_size]
y_train = y[:train_size]
X_test = X[train_size:]
y_test = y[train_size:]

train_size: 168


In [18]:
# Visualize the training data
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_train,
                         y=y_train,
                         mode='markers',
                         line=dict(color='DarkBlue', width=3)))
fig.update_layout(title='Electricity consumption, 2009-2022',
                  xaxis_title='Time',
                  yaxis_title='Consumption',
                  width=700,
                  height=700)
fig.show()


In [19]:
# Fit a simple linear model to identify the trend
model = LinearRegression().fit(np.array(X_train).reshape(-1, 1), y_train)

# Extract coefficients of the linear model
coefficients = [model.coef_[0], model.intercept_]
print("The linear model is F(t) = " + str(coefficients[0]) + "*t + (" + str(coefficients[1]) + ")")

# Predictions using the linear model
linear_predictions = model.predict(np.array(X_train).reshape(-1, 1))

# Visualize the original data and the linear fit
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_train,
                         y=y_train,
                         name='Original data',
                         mode='markers',
                         line=dict(color='DarkBlue', width=3)))
fig.add_trace(go.Scatter(x=X_train,
                         y=linear_predictions,
                         name='Linear fit',
                         line=dict(color='Orange', width=3)))
fig.update_layout(title='Electricity consumption, 2009-2022',
                  xaxis_title='Time',
                  yaxis_title='Consumption',
                  width=700,
                  height=700)
fig.show()

The linear model is F(t) = -1153351.00691432*t + (54667833.72101928)


In [20]:
# Calculate residuals from the linear model
residuals_linear = y_train - linear_predictions

# Visualize the residuals
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_train,
                         y=residuals_linear,
                         mode='markers',
                         line=dict(color='Red', width=3)))
fig.update_layout(title="Residuals (R_linear)",
                  xaxis_title='Time',
                  width=700,
                  height=700)
fig.show()

From the graph we can notice that excluding the seasonal variation there is not a clear systematic trend in residuals, they seem stationary and bounce around the 0 line. Linear model seems a good fit, however, let's also try fitting a quadratic model and compare the results.

In [21]:
# Fit a quadratic model to identify the trend
degree = 2
model_2 = make_pipeline(PolynomialFeatures(degree), LinearRegression())
model_2.fit(np.array(X_train).reshape(-1, 1), y_train)
quadratic_predictions = model_2.predict(np.array(X_train).reshape(-1, 1))

# Extract coefficients of the quadratic model
coefficients_2 = [model_2.named_steps.linearregression.coef_[-1], model_2.named_steps.linearregression.coef_[-2], model_2.named_steps.linearregression.intercept_]
print("The quadratic model is F(t) = " + str(coefficients_2[0]) + "*t^2 + (" + str(coefficients_2[1]) + ")*t + (" + str(coefficients_2[2]) + ")")

# Visualize the original data and the quadratic fit
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_train, y=y_train, name='Original data', mode='markers', line=dict(color='DarkBlue', width=3)))
fig.add_trace(go.Scatter(x=X_train, y=quadratic_predictions, name='Quadratic fit', line=dict(color='Orange', width=3)))
fig.update_layout(title='Electricity consumption, 2009-2022',
                  xaxis_title='Time',
                  yaxis_title='Consumption',
                  width=700,
                  height=700)
fig.show()

The quadratic model is F(t) = -13750.169388872338*t^2 + (-960848.6354701082)*t + (54218653.56370884)


In [22]:
# Calculate residuals from the quadratic model
residuals_quadratic = y_train - quadratic_predictions

# Visualize the residuals for the quadratic model
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_train,
                         y=residuals_quadratic,
                         mode='markers',
                         line=dict(color='Red', width=3)))
fig.update_layout(title="Residuals (R_quadratic)",
                  xaxis_title='Time',
                  width=700,
                  height=700)
fig.show()

The residuals of the quadratic model also doesn't show a clear systematic trend in residuals and they bounce around the 0 line.

Let's compare the RMSE (Root Mean Squared Error) and MAPE (Mean Absolute Percentage Error) errors on the test dataset.

In [23]:
# Predictions on the test dataset for both linear and quadratic models
linear_test_predictions = model.predict(np.array(X_test).reshape(-1, 1))
quadratic_test_predictions = model_2.predict(np.array(X_test).reshape(-1, 1))

# Calculate RMSE for both linear and quadratic models
rmse_linear = mean_squared_error(y_test, linear_test_predictions, squared=False)
rmse_quadratic = mean_squared_error(y_test, quadratic_test_predictions, squared=False)

# Calculate MAPE for both linear and quadratic models
mape_linear = mean_absolute_percentage_error(y_test, linear_test_predictions)
mape_quadratic = mean_absolute_percentage_error(y_test, quadratic_test_predictions)

print("RMSE of the linear model on the test data: " + str(rmse_linear))
print("RMSE of the quadratic model on the test data: " + str(rmse_quadratic))

print("MAPE of the linear model on the test data: " + str(mape_linear))
print("MAPE of the quadratic model on the test data: " + str(mape_quadratic))


RMSE of the linear model on the test data: 4372045.145568254
RMSE of the quadratic model on the test data: 4242848.735712614
MAPE of the linear model on the test data: 10.555465914605293
MAPE of the quadratic model on the test data: 10.072201461326038


We will chose the quadratic model for its slightly superior performance in terms of RMSE and MAPE. To incorporate this model into our dataset, let's add a 'trend' column representing the quadratic predictions. Subsequently, we'll calculate the 'detrended' column by subtracting these predictions from the original consumption values.

In [24]:
# Predictions of the quadratic model on the entire dataset
quadratic_predictions_full = model_2.predict(np.array(X).reshape(-1, 1))

# Update the dataset: add trend column
data.loc[:, 'trend'] = quadratic_predictions_full

# Detrend the consumption column
data.loc[:, 'detrended'] = y - quadratic_predictions_full

data.head()

,month_start,consumption,month,time,trend,detrended
0,2009-01-01,63238939,1,0.041667,5.417859e+07,9.060345e+06
1,2009-02-01,56073378,2,0.125000,5.409833e+07,1.975045e+06
2,2009-03-01,56519297,3,0.208333,5.401788e+07,2.501417e+06
3,2009-04-01,49311414,4,0.291667,5.393724e+07,-4.625822e+06
4,2009-05-01,48437067,5,0.375000,5.385640e+07,-5.419335e+06


Now, we proceed to eliminate the periodic trend from the detrended series. As observed in the initial visualization, the time period for the seasonal variation is 12 months, aligning with the higher electricity consumption during winter months. Initially, we extract the periodic monthly trend by calculating the average trend across all months. Subsequently, we use interpolation between these values to obtain a continuous periodic signal

In [25]:
# Calculate the monthly average of detrended values for the training set
monthly_average = data.iloc[:train_size, :].groupby('month').detrended.mean()

# Generate a continuous set of months for interpolation
month_continuous = np.linspace(1, 12, num=100, endpoint=True)

# Perform quadratic interpolation to obtain a continuous periodic signal
periodic = interp1d(monthly_average.index, monthly_average.values, kind='quadratic')

In [26]:
# Visualize periodic signal
fig = go.Figure()

# Scatter plot for monthly average periodic values
fig.add_trace(go.Scatter(x=monthly_average.index,
                         y=monthly_average.values,
                         mode='markers',
                         line=dict(color='Green', width=7),
                         name='Periodic values'))

# Line plot for the continuous periodic signal
fig.add_trace(go.Scatter(x=month_continuous,
                         y=periodic(month_continuous),
                         line=dict(color='Navy', width=1),
                         name='Periodic signal'))

fig.update_layout(title="Periodic Signal",
                  xaxis_title='Month',
                  width=700,
                  height=700)

fig.show()

In [27]:
# Add periodic signal to the dataframe
data.loc[:, "periodic_signal"] = data['month'].apply(lambda x: periodic(x))

data.head()

,month_start,consumption,month,time,trend,detrended,periodic_signal
0,2009-01-01,63238939,1,0.041667,5.417859e+07,9.060345e+06,8.821133e+06
1,2009-02-01,56073378,2,0.125000,5.409833e+07,1.975045e+06,2.806156e+06
2,2009-03-01,56519297,3,0.208333,5.401788e+07,2.501417e+06,4.263811e+06
3,2009-04-01,49311414,4,0.291667,5.393724e+07,-4.625822e+06,-2.611980e+06
4,2009-05-01,48437067,5,0.375000,5.385640e+07,-5.419335e+06,-3.867877e+06


Now, we'll add both the deterministic and periodic trends to create the final model.

In [30]:
data.loc[:, "final_model"] = data["trend"] + data["periodic_signal"]

data.head()

,month_start,consumption,month,time,trend,detrended,periodic_signal,final_model
0,2009-01-01,63238939,1,0.041667,5.417859e+07,9.060345e+06,8.821133e+06,6.299973e+07
1,2009-02-01,56073378,2,0.125000,5.409833e+07,1.975045e+06,2.806156e+06,5.690449e+07
2,2009-03-01,56519297,3,0.208333,5.401788e+07,2.501417e+06,4.263811e+06,5.828169e+07
3,2009-04-01,49311414,4,0.291667,5.393724e+07,-4.625822e+06,-2.611980e+06,5.132526e+07
4,2009-05-01,48437067,5,0.375000,5.385640e+07,-5.419335e+06,-3.867877e+06,4.998852e+07


In [31]:
# Fizualie the original data and predictions
fig = go.Figure()

# Scatter plot for the training data
fig.add_trace(go.Scatter(x=X_train,
                         y=y_train,
                         mode='markers',
                         name='Training data',
                         line=dict(color='Green', width=3, dash='dot')))

# Line plot for the final model fit on the training data
fig.add_trace(go.Scatter(x=X_train,
                         y=data.loc[data['month_start'] < '2023-01-01', :].final_model,
                         name='Final model (training data fit)',
                         line=dict(color='Red', width=3, dash='dot')))

# Scatter plot for the test data
fig.add_trace(go.Scatter(x=X_test,
                         y=y_test,
                         mode='markers',
                         name='Test data',
                         line=dict(color='Yellow', width=3, dash='dot')))

# Line plot for the final model predictions on the test data
fig.add_trace(go.Scatter(x=X_test,
                         y=data.loc[data['month_start'] >= '2023-01-01', :].final_model,
                         name='Final Model Predictions',
                         line=dict(color='Orange', width=3, dash='dot')))

# Update layout for better presentation
fig.update_layout(title='Electricity consumption, 2009-2023',
                  xaxis_title='Time',
                  yaxis_title='Consumption',
                  width=1400,
                  height=700)

fig.show()

Let's calculate the RMSE and MAPE of the final fit and compare them with the quadratic fit calculated earlier.

In [32]:
# Extract the final model predictions for the test period
final_test = data[data['month_start'] >= '2023-01-01'].final_model

# Calculate RMSE and MAPE for the final model
rmse_final = (mean_squared_error(y_test, final_test)) ** 0.5
mape_final = mean_absolute_percentage_error(y_test, final_test)

print("The Root Mean Squared Error (RMSE) of the final model is " + str(rmse_final))
print("The Mean Absolute Percentage Error (MAPE) of the final model is " + str(mape_final))

The Root Mean Squared Error (RMSE) of the final model is 1066132.9777233147
The Mean Absolute Percentage Error (MAPE) of the final model is 2.613675799991221


We observe a substantial improvement compared to the quadratic model predictions.

While we haven't employed state-of-the-art forecasting methods in this analysis, the compositional approach applied to the 2023 data yielded moderate predictions. The primary focus here was on gaining insights into time series and its fundamental components.

In future analyses, we may explore more advanced forecasting models to enhance prediction accuracy.